In [18]:
# Load necessary libraries
import tkinter as Tk
from PIL import Image, ImageTk, ImageDraw, ImageFont
import os
import numpy as np
import collections
import csv
from random import randint
import pandas as pd

In [10]:
# Specify clip number 
clipNum = 'CLIP_011'

In [11]:
cwd = os.getcwd()
datadir = os.path.join(cwd, 'data', clipNum, 'FRAMES')

# Total number of frames for the clip
n = len(os.listdir(datadir))

positions = {}

# Default size for circle for tracking
radius = 20

In [5]:
# Handle mouse click
def onButtonPress(event):
    canvas = event.widget

    x = canvas.canvasx(event.x)
    y = canvas.canvasy(event.y)
    
    # If there is already a circle/guess, move 
    if len(canvas.find_all()) == 2:
        oldPos = canvas.coords(2)
        canvas.move(2, x-radius-oldPos[0], y-radius-oldPos[1])
   
    # Otherwise, draw a new circle
    else:
        circle = canvas.create_oval(x-radius, y-radius, 
                                    x+radius, y+radius,
                                    outline="#f11", width=2)

# Handle closing out 
def onEsc(event, root, imageNum):
    global positions
    canvas = event.widget
    
    # Save coordinates into dict with frame number as key and coordinates as value
    positions[imageNum] = canvas.coords(2)
    
    root.destroy()
    
# Show the first frame again, highlight who is being tracked
def showFirst(event):
    path = os.path.join(datadir, 'frame_%04d.png' % (start))
    root = Tk.Tk()

    canvas = Tk.Canvas(root, cursor="cross")
    canvas.pack(fill=Tk.BOTH, expand=1) 
    canvas.focus_set()
    canvas.bind("<h>", lambda event: destroyFirst(event, root))

    background_image = ImageTk.PhotoImage(Image.open(path).resize((1200,700)), master=root)
    image = canvas.create_image(0, 0, anchor=Tk.NW, image=background_image)

    
    # Get saved location for the first frame and show on image
    guess = positions[start]
    circle = canvas.create_oval(guess[0],guess[1],guess[2],guess[3],
                               outline="#f11", width=2)
    root.wm_geometry("1200x700")
    root.title('First frame')
    root.mainloop()
                

# Close out the first image
def destroyFirst(event, root):
    root.destroy()
    

# Given some frame number, get the image and show it 
# If there is a guess for coordinates, get from dict and show on image
def showImage(imageNum):
    root = Tk.Tk()

    canvas = Tk.Canvas(root, cursor="cross")
    canvas.pack(fill=Tk.BOTH, expand=1) # Stretch canvas to root window size.
    canvas.focus_set()
    
    # Mouse click to specify location
    canvas.bind("<Button-1>", onButtonPress)
    # Space button to move onto next frame
    canvas.bind("<space>", lambda event: onEsc(event, root, imageNum))
    # 'h' key to show/hide the first frame (to check who is being tracked)
    canvas.bind("<h>", lambda event: showFirst(event))
    
    if started and imageNum == start:
        path = os.path.join('data/'+ clipNum +'/TRACKED_FRAMES', 'frame_%04d.png' % (imageNum))
    else:
        path = os.path.join(datadir, 'frame_%04d.png' % (imageNum))

        
    background_image = ImageTk.PhotoImage(Image.open(path).resize((1200,700)))
    image = canvas.create_image(0, 0, anchor=Tk.NW, image=background_image)
    
    if imageNum in positions:
        guess = positions[imageNum]
        circle = canvas.create_oval(guess[0],guess[1],guess[2],guess[3],
                                    outline="#f11", width=2)
    root.wm_geometry("1200x700")
    root.title('Frame %d' % imageNum)
    root.mainloop()

In [6]:
# Given two time points and location coords for each of those time points, 
# get the location at the middle time point (based on linear interpolation)
# Return this "guessed" location
def interpolate(t1, t2):
    global positions
    guess = list(np.add(positions[t1], positions[t2])//2)
    positions[(t2+t1)//2] = guess
    return guess

In [7]:
# Recursively check the middle guesses of the two time points given
# If the location of the mid point guess is incorrect, update the midpoint location
# and continue to check the midpoints
def checkMiddle(t1, t2):
    count = 0
    m = (t1 + t2)//2
    if m == t1: return count#t1, t2 are consecutive frames
    #print ("checking " + str(m))
    
    tmp = interpolate(t1, t2)
    showImage(m)
    
    if positions[m] != tmp:
        count += 1
        if (m == (t1 + 1)): 
            return count #t1, t2 are 2 apart
        
        count += checkMiddle(t1, m)
        count += checkMiddle(m, t2)
    
    return count 

In [1]:
# Given some sparsely filled dictionary with all known locations and time points
# Do the linear interpolation and get the exact location at every frame
def backfillPositions(positions, first, last):
    #after tracking, fill in the rest of the positions
    fullPositions = positions.copy()
    od = collections.OrderedDict(sorted(positions.items()))
    for frame, pos in od.items(): 
        # If we are looking at the first frame, note that it is the last known location
        if frame == first: 
            lastFrame = first
            continue
        
        # If the last known location was the immediately previous frame, no need for interpolating
        if lastFrame + 1 == frame:
            lastFrame = frame
            continue
            
        # Otherwise, do linear interpolation to get the exact location for all the frames between
        # the previous known location and the current known location
        else: 
            stepSize = np.subtract(pos, positions[lastFrame])/(frame - lastFrame)
            for i in range(lastFrame + 1, frame):
                fullPositions[i] = list(np.add(stepSize, fullPositions[i-1]))
            lastFrame = frame
            
    # If started tracking after the first frame, fill in all frames prior to start as out of bounds
    if first > 1: 
        for i in range(1, first):
            fullPositions[i] = [1166.0, 656.0, 1206.0, 696.0]
    
    # If ended tracking before the last frame, fill in all frames after ending as out of bounds
    if end < n: 
        for i in range(last + 1, n+1):
            fullPositions[i] = [1166.0, 656.0, 1206.0, 696.0]
    
    fullSorted = collections.OrderedDict(sorted(fullPositions.items()))
    return fullSorted

In [9]:
# Save data from dictionary into .csv
# Each column is a frame and each row is a new person
def saveData(datafile, subID, positions):
    with open (datafile, 'a') as csvfile: 
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow([])
        data = [subID]
        for frame, pos in positions.items():
            data.append(pos)
        writer.writerow(data)

In [17]:
# Just get the data for the first frame and draw the locations of everyone who has been tracked
def saveFirstFrame(sourceDir, destDir, positionsFile, start):
    df = pd.read_csv(positionsFile, header=None)
    df.dropna(inplace=True)
    colors = [(195, 195, 195) for x in df.index]
    font = ImageFont.truetype("VCR_OSD_MONO_1.001.ttf", 16)
    #font = ImageFont.load_default()
    for col in df.columns[start:start+1]: 
        path = os.path.join(sourceDir, 'frame_%04d.png' % (col))
        newPath = os.path.join(destDir, 'frame_%04d.png' % (col))

        img = Image.open(path, 'r')
        img = img.resize((1200,700))
        img_w, img_h = img.size
        background = Image.new('RGBA', (img_w, img_h), (255, 255, 255, 255))
        background.paste(img, (0,0))
        draw = ImageDraw.Draw(background)
        draw.text((20,20), str(col), (255,0,0), font=font)
        
        # For everyone tracked thus far, show their location on the image
        for ID, person in enumerate(df[col]):
            coords = [float(x) for x in person[1:-1].split(',')]
            draw.ellipse(coords, outline=colors[ID], width=2)
            draw.text((coords[0], coords[1]), str(ID), colors[ID], font=font)
        background.save(newPath)

___
## DO NOT EDIT ABOVE 
___

In [13]:
### PARAMETERS 

# If at least one person in clip has been tracked, set to True
started = True

In [15]:
# Desired frame interval for tracking 
# Default is start = 1 and end = n (total number of frames)

start = 1
end = 416

In [ ]:
# Doing the tracking 
positions = {}

if started:
    saveFirstFrame('data/' + clipNum + '/FRAMES', 
                   'data/'+ clipNum +'/TRACKED_FRAMES', 
                   'data/'+ clipNum +'/DATA/positions.csv',
                   start)

showImage(start)
showImage(end)

numEdits = checkMiddle(start, end)
fullTest = backfillPositions(positions, start, end)
print (numEdits)

In [48]:
# Saving position data to .csv in clip's DATA subdirectory
saveData('data/' + clipNum + '/DATA/positions.csv', numEdits, fullTest)